In [ ]:
import os 
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from agent_framework import AgentRunResponse, ChatResponseUpdate, HostedCodeInterpreterTool ,
from agent_framework import HostedMCPTool
from agent_framework import ChatAgent,ChatMessage
from azure.ai.projects.aio import AIProjectClient
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio  import AzureCliCredential



In [ ]:
load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

In [ ]:
code_interpreter_tool = HostedCodeInterpreterTool()


In [ ]:
ms_learn_mcp_tool = HostedMCPTool(
    name = "Microsoft Learn MCP Tool",
    url = "https://learn.microsoft.com/api/mcp"
)

In [ ]:
async def create_multi_tool_agent():
    credential = AzureCliCredential()
    chat_client =AzureAIAgentClient(
        async_credential= credential,
        project_endpoint= project_endpoint,
        model_deployment_name=model
    )
    agent = chat_client.create_agent(
        name = "multi-agent-tool",
        instructions= "You are helpful multi tool agent",
        tools = [code_interpreter_tool, ms_learn_mcp_tool]
    )
    return agent 
    


In [ ]:
agent = await create_multi_tool_agent()

In [ ]:
async def handle_approvals_with_thread(query : str, agent, thread):
    result =  await agent.run(query,thread =  thread,store = True)
    while len(result.user_input_requests) > 0 :
        new_input = []
        for user_input_needed in result.user_input_requests :
            new_input.append(
                ChatMessage(
                    role = "user",
                    content = [ user_input_needed.create_response(True)]

                )
            )
        result = await agent.run(new_input, thread= thread, store = True)
    return result 

In [ ]:
thread =  agent.get_new_thread()
query = "Find me information about Azure Functions in Microsoft Learn."
response = await handle_approvals_with_thread(query, agent, thread)

print("\n Assistant Response:", response)